In [22]:
from pathlib import Path
import pandas as pd

# Edge mappings are only based on isomorphic graphs from the en gold train dataset
PATH = Path('../data/edge_classifier/edge_mappings_2022_05_10_17_06_42.csv')
df = pd.read_csv(PATH)

<ipython-input-22-3bae70ba7f33>:6: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH)


In [23]:
df.deprel.unique()

array([nan, 'nsubj', 'obj', 'advmod', 'obl', 'amod', 'nsubj:pass', 'iobj',
       'nummod', 'conj', 'obl:npmod', 'nmod:poss', 'nmod', 'csubj',
       'compound', 'nmod:tmod', 'xcomp', 'acl', 'obl:tmod', 'expl',
       'ccomp', 'advcl', 'appos', 'flat', 'vocative', 'nmod:npmod',
       'det:predet', 'parataxis', 'acl:relcl', 'cc', 'det', 'case'],
      dtype=object)

In [24]:
# Drop nodes without pos tags
df = df[(df.from_node_upos.notnull()) & (df.to_node_upos.notnull()) & (df.deprel.notnull())]

In [25]:
df

,from_node_token,from_node_Case,from_node_Number,from_node_Person,from_node_PronType,from_node_lemma,from_node_upos,from_node_xpos,to_node_token,deprel,...,from_node_Degree,to_node_Poss,from_node_Poss,to_node_Tense,to_node_VerbForm,from_node_Gender,to_node_Mood,to_node_Voice,to_node_Reflex,to_node_Foreign
2,joke.v.01,NaN,NaN,NaN,NaN,joke,VERB,VBG,person.n.01,nsubj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,hate.v.01,NaN,Sing,3.0,NaN,hate,VERB,VBD,male.n.02,obj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,hate.v.01,NaN,Sing,3.0,NaN,hate,VERB,VBD,female.n.02,nsubj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,videotape.v.01,NaN,NaN,NaN,NaN,videotape,VERB,VBD,female.n.02,nsubj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,videotape.v.01,NaN,NaN,NaN,NaN,videotape,VERB,VBD,person.n.01,obj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30338,love.v.01,NaN,NaN,NaN,NaN,love,VERB,VBP,person.n.01,nsubj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30339,love.v.01,NaN,NaN,NaN,NaN,love,VERB,VBP,female.n.02,obj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30344,work.v.02,NaN,NaN,NaN,NaN,work,VERB,VBG,television.n.03,nsubj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30348,sleep.v.01,NaN,NaN,NaN,NaN,sleep,VERB,VBG,person.n.01,nsubj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
def create_key(from_upos, deprel, to_upos):
    return '-'.join([from_upos, deprel, to_upos])

mapping = dict()
for _, row in df.iterrows():
    label = row.label
    key = create_key(row.from_node_upos, row.deprel, row.to_node_upos)
    if key in mapping:
        if label in mapping[key]:
            mapping[key][label] += 1
        else:
            mapping[key][label] = 1
    else:
        mapping[key] = {label: 1}

In [32]:
def filter_most_common(mapping_dict):
    filtered_dict = dict()
    for lem, count_dict in mapping_dict.items():
        if len(count_dict) == 1:
            filtered_dict[lem] = list(count_dict.keys())[0]
        else:
            filtered_dict[lem] = sorted(list(count_dict.items()), key=lambda x: x[1])[-1][0]
    return filtered_dict

In [35]:
most_common = filter_most_common(mapping)

In [37]:
import json

with open('edge_mappings_train.json', 'w') as f:
    json.dump(most_common, f, indent=2)